In [3]:
import sys
sys.path.append("../../src")
sys.path.append("../..")
import os
import datetime
import pandas as pd
import numpy as np
from example_thomas import get_thomas_data
from sindy_utils import library_size
from training import train_network
#import tensorflow as tf
import tensorflow.compat.v1 as tf


In [4]:
!pwd 
!ls ../..

/c/Users/Aygalic/OneDrive/polimi/COURSES/S10/Computational Stats/SindyAutoencoders_v2/examples/thomas
LICENSE
README.md
examples
nate
rd_solver
robi
src


# Generate data

In [5]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_thomas_data(1024, noise_strength=noise_strength)
validation_data = get_thomas_data(50, noise_strength=noise_strength)

# Set up model and training parameters

In [6]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 2
params['include_sine'] = True
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100
params['print_frequency'] = 1 # for testing

# training time cutoffs
params['max_epochs'] = 5001 # WAY TOO LONG ON AN M1 MAX
params['max_epochs'] = 1000
params['refinement_epochs'] = 1001 # idk what this is 
params['refinement_epochs'] = 100

# Run training experiments

In [7]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
TRAINING
Epoch 0
   training loss 0.27228760719299316, (0.27212873, 285.6557, 1.5222338, 0.6643909)
   validation loss 0.2891254723072052, (0.28902632, 488.42545, 0.92515945, 0.6643909)
decoder loss ratio: 0.602959, decoder SINDy loss  ratio: 59.123707
Epoch 1
   training loss 0.16994956135749817, (0.16990462, 6.6365457, 0.40834934, 0.41062605)
   validation loss 0.1761362999677658, (0.17609882, 6.762764, 0.3335649, 0.41062605)
decoder loss ratio: 0.367372, decoder SINDy loss  ratio: 21.316967
Epoch 2
   training loss 0.1638975888490677, (0.16387555, 1.9116591, 0.19271144, 0.2775983)
   validation loss 0.14206191897392273, (0.14204235, 1.9624577, 0.1678712, 0.2775983)
decoder loss ratio: 0.296325, decoder SINDy loss  ratio: 10.728062
Epoch 3
   training loss 0.15845392644405365, (0.15844516, 0.3506659, 0.07081626, 0.1678691)
   validation loss 0.1327546238899231, (0.132746, 0.5718062, 0.069484

Epoch 31
   training loss 0.0016334877582266927, (0.0016331384, 0.002683928, 0.0017619537, 0.017311892)
   validation loss 0.002950007561594248, (0.0029495426, 0.0034150444, 0.002917429, 0.017311892)
decoder loss ratio: 0.006153, decoder SINDy loss  ratio: 0.186443
Epoch 32
   training loss 0.0014715883880853653, (0.0014712412, 0.0026351402, 0.0017373576, 0.017347202)
   validation loss 0.0027098844293504953, (0.0027094223, 0.0036034689, 0.0028880048, 0.017347202)
decoder loss ratio: 0.005652, decoder SINDy loss  ratio: 0.184562
Epoch 33
   training loss 0.0013592859031632543, (0.0013589419, 0.002576152, 0.0017148028, 0.017250264)
   validation loss 0.0025106805842369795, (0.0025102212, 0.0037836365, 0.0028682186, 0.017250264)
decoder loss ratio: 0.005237, decoder SINDy loss  ratio: 0.183298
Epoch 34
   training loss 0.00127892242744565, (0.0012785788, 0.0025363506, 0.0017086302, 0.017277883)
   validation loss 0.0023451673332601786, (0.0023447084, 0.0039459243, 0.0028611114, 0.0172778

Epoch 62
   training loss 0.0004005418159067631, (0.0004002385, 0.0016408786, 0.0013921118, 0.016411036)
   validation loss 0.0006430043140426278, (0.0006426162, 0.00330865, 0.0022405083, 0.016411036)
decoder loss ratio: 0.001341, decoder SINDy loss  ratio: 0.143183
Epoch 63
   training loss 0.0004049435956403613, (0.00040464065, 0.0016130977, 0.0013874376, 0.016420456)
   validation loss 0.0006324643618427217, (0.0006320775, 0.0032793132, 0.0022266388, 0.016420456)
decoder loss ratio: 0.001319, decoder SINDy loss  ratio: 0.142297
Epoch 64
   training loss 0.00043819600250571966, (0.00043789533, 0.0015707975, 0.0013660123, 0.016405582)
   validation loss 0.0006410006899386644, (0.0006406173, 0.0031920155, 0.0021935932, 0.016405582)
decoder loss ratio: 0.001336, decoder SINDy loss  ratio: 0.140185
Epoch 65
   training loss 0.0005424459232017398, (0.0005421475, 0.0015432759, 0.0013450837, 0.016388975)
   validation loss 0.0006732106558047235, (0.0006728297, 0.0031547344, 0.0021704657, 0.

Epoch 93
   training loss 0.0003399705747142434, (0.0003396821, 0.001380661, 0.0012647054, 0.016203502)
   validation loss 0.0005037631490267813, (0.0005033985, 0.0028320984, 0.0020263437, 0.016203502)
decoder loss ratio: 0.001050, decoder SINDy loss  ratio: 0.129497
Epoch 94
   training loss 0.00018524059851188213, (0.00018494835, 0.0014109627, 0.001298098, 0.016244145)
   validation loss 0.0003785214794334024, (0.0003781538, 0.0027878096, 0.0020525407, 0.016244145)
decoder loss ratio: 0.000789, decoder SINDy loss  ratio: 0.131171
Epoch 95
   training loss 0.00016503449296578765, (0.00016474638, 0.0013412753, 0.0012460273, 0.016350707)
   validation loss 0.00030719494679942727, (0.0003068306, 0.0027704842, 0.0020084532, 0.016350707)
decoder loss ratio: 0.000640, decoder SINDy loss  ratio: 0.128353
Epoch 96
   training loss 0.00017854210454970598, (0.0001782547, 0.0013519702, 0.0012475624, 0.016264807)
   validation loss 0.0003393929509911686, (0.00033903084, 0.002763454, 0.001994363, 

Epoch 124
   training loss 0.0001751979871187359, (0.00017492141, 0.0012871854, 0.0011450204, 0.016206995)
   validation loss 0.0003491000388748944, (0.00034874986, 0.0027069747, 0.0018810125, 0.016206995)
decoder loss ratio: 0.000728, decoder SINDy loss  ratio: 0.120209
Epoch 125
   training loss 8.081541454885155e-05, (8.053539e-05, 0.0013659237, 0.0011808563, 0.01619332)
   validation loss 0.000219969660975039, (0.00021961777, 0.0026947688, 0.0018995398, 0.01619332)
decoder loss ratio: 0.000458, decoder SINDy loss  ratio: 0.121393
Epoch 126
   training loss 5.396838241722435e-05, (5.3688404e-05, 0.0013477448, 0.0011826743, 0.016171323)
   validation loss 0.00017526275769341737, (0.00017491255, 0.0026780749, 0.0018849337, 0.016171323)
decoder loss ratio: 0.000365, decoder SINDy loss  ratio: 0.120460
Epoch 127
   training loss 9.312549809692428e-05, (9.284823e-05, 0.0012802759, 0.0011602284, 0.016124392)
   validation loss 0.0002555101236794144, (0.0002551613, 0.0026428462, 0.00187582

...

Epoch 961
   training loss 7.077118789311498e-05, (7.028488e-05, 0.016856615, 0.0035676647, 0.012954316)
   validation loss 0.00020144674635957927, (0.00020086562, 0.019541439, 0.0045159324, 0.012954316)
decoder loss ratio: 0.000419, decoder SINDy loss  ratio: 0.288597
Epoch 962
   training loss 4.952888411935419e-05, (4.9043963e-05, 0.016927524, 0.0035520617, 0.012971573)
   validation loss 0.0001814115239540115, (0.00018083068, 0.019902848, 0.0045112004, 0.012971573)
decoder loss ratio: 0.000377, decoder SINDy loss  ratio: 0.288295
Epoch 963
   training loss 0.00010958638449665159, (0.00010910274, 0.016502991, 0.0035407643, 0.012956977)
   validation loss 0.0002478067472111434, (0.00024722918, 0.019862771, 0.0044799224, 0.012956977)
decoder loss ratio: 0.000516, decoder SINDy loss  ratio: 0.286296
Epoch 964
   training loss 0.0001289688516408205, (0.00012848427, 0.01607914, 0.003548788, 0.012970628)
   validation loss 0.0002676667645573616, (0.00026708594, 0.019397719, 0.0045109917, 

Epoch 992
   training loss 5.362470983527601e-05, (5.313055e-05, 0.01824542, 0.003648533, 0.012930961)
   validation loss 0.00015115426504053175, (0.00015056762, 0.02084017, 0.0045733107, 0.012930961)
decoder loss ratio: 0.000314, decoder SINDy loss  ratio: 0.292264
Epoch 993
   training loss 0.00013625595602206886, (0.00013576451, 0.016986804, 0.0036220793, 0.0129239485)
   validation loss 0.0002505746961105615, (0.00024998933, 0.020325983, 0.0045610387, 0.0129239485)
decoder loss ratio: 0.000522, decoder SINDy loss  ratio: 0.291480
Epoch 994
   training loss 5.381904702517204e-05, (5.3325846e-05, 0.018120186, 0.003638529, 0.012934969)
   validation loss 0.00015239068306982517, (0.00015180402, 0.020890595, 0.0045731496, 0.012934969)
decoder loss ratio: 0.000317, decoder SINDy loss  ratio: 0.292254
Epoch 995
   training loss 0.0001536383351776749, (0.00015314693, 0.016867, 0.0036224648, 0.012915807)
   validation loss 0.00026438714121468365, (0.00026380143, 0.020347528, 0.0045655267, 0

Epoch 23
   training loss 0.00012722449901048094, (0.00012687413, 0.019493781, 0.003503655, 0.0140099935)
   validation loss 0.00024971223319880664, (0.00024926683, 0.022251332, 0.004454082, 0.0140099935)
decoder loss ratio: 0.000520, decoder SINDy loss  ratio: 0.284645
Epoch 24
   training loss 6.472035602200776e-05, (6.437367e-05, 0.018194681, 0.0034668325, 0.014003969)
   validation loss 0.00016672993660904467, (0.00016628794, 0.021519452, 0.004419954, 0.014003969)
decoder loss ratio: 0.000347, decoder SINDy loss  ratio: 0.282464
Epoch 25
   training loss 3.757966260309331e-05, (3.7233207e-05, 0.018912062, 0.0034645717, 0.013986602)
   validation loss 0.00014077327796258032, (0.00014033279, 0.021986887, 0.004404853, 0.013986602)
decoder loss ratio: 0.000293, decoder SINDy loss  ratio: 0.281499
Epoch 26
   training loss 7.541706872871146e-05, (7.506898e-05, 0.018720757, 0.0034808842, 0.013973369)
   validation loss 0.00017577459220774472, (0.00017533134, 0.022181489, 0.0044325152, 0.

Epoch 54
   training loss 0.00017662844038568437, (0.00017627845, 0.018549202, 0.0034998353, 0.013945859)
   validation loss 0.0002749658888205886, (0.00027451935, 0.022388533, 0.0044654314, 0.013945859)
decoder loss ratio: 0.000573, decoder SINDy loss  ratio: 0.285370
Epoch 55
   training loss 7.092544547049329e-05, (7.057334e-05, 0.02025237, 0.003521026, 0.01396301)
   validation loss 0.00017864047549664974, (0.00017819421, 0.023351964, 0.00446267, 0.01396301)
decoder loss ratio: 0.000372, decoder SINDy loss  ratio: 0.285194
Epoch 56
   training loss 0.00012340852117631584, (0.00012305788, 0.019421773, 0.003506431, 0.013948353)
   validation loss 0.00023382359358947724, (0.00023337768, 0.022973314, 0.004459184, 0.013948353)
decoder loss ratio: 0.000487, decoder SINDy loss  ratio: 0.284971
Epoch 57
   training loss 5.962340219411999e-05, (5.927505e-05, 0.019737776, 0.0034835138, 0.013939993)
   validation loss 0.00016524949751328677, (0.00016480456, 0.0232226, 0.00444943, 0.013939993)

Epoch 85
   training loss 7.925214595161378e-05, (7.89013e-05, 0.020720981, 0.0035084495, 0.013897465)
   validation loss 0.00019543463713489473, (0.00019498722, 0.023853041, 0.0044741947, 0.013897465)
decoder loss ratio: 0.000407, decoder SINDy loss  ratio: 0.285930
Epoch 86
   training loss 9.302052058046684e-05, (9.266648e-05, 0.019780472, 0.0035403748, 0.013899374)
   validation loss 0.00018439869745634496, (0.00018394933, 0.023351094, 0.004493578, 0.013899374)
decoder loss ratio: 0.000384, decoder SINDy loss  ratio: 0.287169
Epoch 87
   training loss 7.509413262596354e-05, (7.474366e-05, 0.020679984, 0.003504763, 0.013894804)
   validation loss 0.00018925701442640275, (0.00018880994, 0.023854509, 0.004470847, 0.013894804)
decoder loss ratio: 0.000394, decoder SINDy loss  ratio: 0.285716
Epoch 88
   training loss 9.459102875553071e-05, (9.4237184e-05, 0.019665152, 0.003538448, 0.013895712)
   validation loss 0.0001822453923523426, (0.00018179612, 0.023263134, 0.004492712, 0.0138957